## Mask R-CNN Model

In [1]:
import cv2
import numpy as np
import colorsys
import time
from time import time as timer

cv2.__version__

'3.4.0'

In [2]:
import os
import sys
#import random
#import math
#import numpy as np
#import skimage.io
import matplotlib
import matplotlib.pyplot as plt
    
# Root directory of the project
ROOT_DIR = os.path.abspath("../")
print(ROOT_DIR)
    
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
import mrcnn.utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline

/home/cfchen/Github36/Mask_RCNN-master


/home/cfchen/anaconda3/envs/MaskRCNN/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

## Directory of images to run detection on
#IMAGE_DIR = os.path.join(ROOT_DIR, "images")
    
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()
    
print(ROOT_DIR)


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [123.7 116.8 103.9

## Configurations

## Load Model

In [4]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

In [5]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

print(ROOT_DIR)

/home/cfchen/Github36/Mask_RCNN-master


## Functions

In [6]:
#############
# Functions #
#############

def random_colors(N, bright=True):
    """
    Generate random colors.
    To get visually distinct colors, generate them in HSV space then
    convert to RGB.
    """
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    np.random.shuffle(colors)
    return colors

def random_colors(N):
    np.random.seed(1)
    colors = [tuple(255 * np.random.rand(3)) for _ in range(N)]
    return colors

colors = random_colors(len(class_names))
class_dict = {
    name: color for name, color in zip(class_names, colors)
}

def apply_mask(image, mask, color, alpha=0.5):
    """apply mask to image"""
    for n, c in enumerate(color):
        image[:, :, n] = np.where(
            mask == 1,
            image[:, :, n] * (1 - alpha) + alpha * c,
            image[:, :, n]
        )
    return image



### Masked

In [22]:
def display_instances(image, boxes, masks, ids, names, scores):
    """
        take the image and results and apply the mask, box, and Label
    """
    n_instances = boxes.shape[0]
    #print(n_instances)
    #print('Frame Number {}: Found {} boxes for {}'.format(count - 1, n_instances, 'img'))

    if not n_instances:
        print('NO INSTANCES TO DISPLAY')
    else:
        assert boxes.shape[0] == masks.shape[-1] == ids.shape[0]

    for i in range(n_instances):
        '''
        Detect Only Person
        '''
        '''        
        if ids[i] != 1:
            continue
        '''
        '''
        Detect Only Person
        '''        
        if not np.any(boxes[i]):
            continue

        y1, x1, y2, x2 = boxes[i]
        label = names[ids[i]]
        color = class_dict[label]
        score = scores[i] if scores is not None else None
        caption = '{} {:.2f}'.format(label, score) if score else label
        mask = masks[:, :, i]

        image = apply_mask(image, mask, color)
        image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        #image = cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255,0), 5)
        #image = cv2.putText(image, caption, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.7, color, 2)
        image = cv2.putText(image, caption, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 255, 255), 2)
        
        '''
        #print(n_instances)
        TLBR = '{} {} {} {}'.format(x1, y1, x2, y2)
        captionLaIdSc = '{} {} {:.2f}'.format(label, ids[i], score) if score else label        
        print("{}  {}  {}".format(count - 1 , captionLaIdSc, TLBR))
        #print(ids)
        #print(ids[i])
        '''
        #print(n_instances)
        TLBR = '{}, {}, {}, {}'.format(x1, y1, x2, y2)
        captionLaIdSc = '{}, {}, {:.2f},'.format(label, ids[i], score) if score else label        
        print("{},  {}  {}".format(count - 1 , captionLaIdSc, TLBR))
        
    return image

capture = cv2.VideoCapture('/media/cfchen/956df7bc-562e-4f24-8339-fd0b67f98888/Downloaded/VideosHPB/IMAG0011.mp4')

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('temp.avi',fourcc, 30.0, (960, 540))     # resize image half

print("Video Dim: {}  {}  {}".format(capture.get(3), 'X', capture.get(4)))

start = timer()

count = 0
while(True):
    ret, frame = capture.read() 
    
    if ret == True:

        count += 1
        
        frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)  # resize image half

        results = model.detect([frame], verbose=0)

        r = results[0]
    
        frame = display_instances(frame, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])

        '''
        print('Frame NO.    {}: Found {} boxes for {}'.format(count - 1, r['rois'].shape[0], 'img'))
        #print('{}  {}  {}'.format(print(r['scores']), r['class_ids'], r['rois']))
        #print(r['scores'])           # Confidence
        #print(r['rois'])             # Bounding Boxes 
        #print(r['class_ids'])        # Class IDs        
        '''
        
        out.write(frame)
        
        cv2.imshow('frame', frame)
        
        if (cv2.waitKey(1) & 0xFF == ord('q')) or (ret == False):
            break

    else:
        break
        
print('++++++++ Video End ++++++++')
  
capture.release()
out.release()
cv2.destroyAllWindows()

print('\nTotal elapsed time = ' + str(timer() - start) + ' s\n')

Video Dim: 1920.0  X  1080.0
0,  car, 3, 0.97,  533, 47, 577, 86
0,  potted plant, 59, 0.93,  42, 3, 229, 125
0,  car, 3, 0.90,  447, 61, 516, 129
0,  truck, 8, 0.86,  142, 94, 262, 179
0,  car, 3, 0.79,  50, 178, 162, 267
0,  car, 3, 0.74,  224, 56, 332, 121
0,  car, 3, 0.72,  124, 333, 199, 407
0,  person, 1, 0.71,  318, 92, 334, 122
1,  car, 3, 0.99,  533, 48, 576, 87
1,  car, 3, 0.98,  447, 65, 514, 129
1,  potted plant, 59, 0.97,  39, 4, 231, 117
1,  car, 3, 0.88,  50, 179, 161, 268
1,  truck, 8, 0.84,  144, 93, 264, 179
1,  person, 1, 0.83,  318, 93, 334, 122
1,  car, 3, 0.77,  117, 356, 176, 413
1,  car, 3, 0.73,  714, 178, 778, 268
1,  car, 3, 0.70,  639, 39, 663, 70
2,  car, 3, 0.99,  532, 53, 574, 88
2,  potted plant, 59, 0.96,  35, 8, 227, 119
2,  car, 3, 0.96,  445, 68, 514, 132
2,  truck, 8, 0.92,  140, 93, 265, 179
2,  car, 3, 0.87,  51, 180, 161, 269
2,  car, 3, 0.82,  639, 35, 674, 71
2,  car, 3, 0.78,  118, 365, 173, 413
2,  truck, 8, 0.78,  201, 27, 331, 125
2,  car, 

24,  car, 3, 0.97,  505, 63, 565, 115
24,  car, 3, 0.96,  640, 40, 668, 70
24,  potted plant, 59, 0.92,  39, 4, 240, 119
24,  car, 3, 0.88,  653, 32, 691, 73
24,  truck, 8, 0.86,  143, 90, 265, 177
24,  car, 3, 0.86,  52, 178, 163, 267
24,  truck, 8, 0.83,  203, 25, 335, 124
24,  truck, 8, 0.78,  375, 90, 471, 185
24,  car, 3, 0.77,  637, 40, 653, 60
24,  car, 3, 0.75,  562, 8, 582, 22
24,  person, 1, 0.71,  317, 92, 337, 122
24,  car, 3, 0.70,  583, 2, 602, 17
25,  car, 3, 0.96,  640, 40, 668, 70
25,  potted plant, 59, 0.92,  40, 3, 239, 119
25,  car, 3, 0.90,  504, 63, 563, 117
25,  car, 3, 0.88,  653, 32, 691, 73
25,  truck, 8, 0.87,  370, 92, 468, 189
25,  car, 3, 0.87,  51, 178, 162, 266
25,  truck, 8, 0.85,  144, 90, 265, 177
25,  truck, 8, 0.85,  203, 25, 334, 124
25,  car, 3, 0.75,  638, 40, 653, 60
25,  car, 3, 0.75,  561, 8, 582, 22
25,  person, 1, 0.71,  317, 92, 337, 122
26,  car, 3, 0.96,  640, 39, 668, 70
26,  potted plant, 59, 0.92,  40, 4, 240, 119
26,  car, 3, 0.87,  6

46,  potted plant, 59, 0.94,  42, 1, 228, 122
46,  car, 3, 0.93,  639, 36, 671, 70
46,  bus, 6, 0.89,  251, 141, 399, 289
46,  truck, 8, 0.88,  148, 90, 262, 178
46,  car, 3, 0.83,  651, 31, 692, 74
46,  car, 3, 0.82,  461, 84, 539, 160
46,  truck, 8, 0.81,  206, 26, 334, 123
46,  car, 3, 0.79,  561, 8, 582, 23
46,  car, 3, 0.75,  52, 179, 163, 267
46,  car, 3, 0.72,  583, 2, 601, 17
47,  car, 3, 0.95,  455, 85, 540, 163
47,  car, 3, 0.94,  640, 38, 669, 70
47,  potted plant, 59, 0.90,  39, 2, 240, 118
47,  truck, 8, 0.86,  148, 90, 261, 178
47,  car, 3, 0.86,  650, 31, 692, 73
47,  truck, 8, 0.81,  206, 26, 334, 123
47,  car, 3, 0.80,  561, 8, 582, 23
47,  car, 3, 0.80,  53, 179, 162, 267
47,  truck, 8, 0.79,  245, 148, 391, 292
48,  car, 3, 0.94,  640, 38, 671, 70
48,  potted plant, 59, 0.93,  42, 0, 239, 129
48,  truck, 8, 0.87,  204, 26, 333, 123
48,  car, 3, 0.85,  560, 9, 582, 25
48,  truck, 8, 0.84,  148, 90, 261, 177
48,  car, 3, 0.82,  583, 2, 602, 17
48,  car, 3, 0.81,  51, 1

66,  car, 3, 0.96,  640, 39, 668, 70
66,  bus, 6, 0.96,  86, 226, 284, 448
66,  car, 3, 0.94,  379, 120, 502, 251
66,  potted plant, 59, 0.93,  44, 0, 229, 135
66,  car, 3, 0.88,  654, 34, 689, 73
66,  truck, 8, 0.86,  150, 89, 267, 175
66,  truck, 8, 0.82,  205, 26, 332, 124
66,  car, 3, 0.81,  562, 8, 581, 22
66,  person, 1, 0.78,  317, 91, 337, 122
66,  car, 3, 0.77,  50, 178, 163, 267
66,  car, 3, 0.75,  583, 3, 600, 17
66,  car, 3, 0.73,  403, 37, 444, 62
66,  car, 3, 0.73,  683, 28, 741, 88
67,  car, 3, 0.97,  377, 126, 499, 248
67,  car, 3, 0.96,  640, 39, 669, 71
67,  potted plant, 59, 0.92,  47, 3, 220, 158
67,  car, 3, 0.89,  652, 34, 689, 73
67,  truck, 8, 0.83,  150, 90, 263, 177
67,  truck, 8, 0.82,  205, 26, 332, 124
67,  car, 3, 0.81,  560, 9, 581, 28
67,  person, 1, 0.79,  317, 92, 337, 122
67,  truck, 8, 0.79,  75, 214, 271, 465
67,  car, 3, 0.77,  50, 178, 163, 266
67,  car, 3, 0.75,  583, 3, 600, 17
67,  car, 3, 0.74,  404, 37, 444, 62
68,  car, 3, 0.96,  640, 38, 66

84,  potted plant, 59, 0.92,  37, 2, 243, 117
84,  car, 3, 0.91,  640, 36, 671, 71
84,  car, 3, 0.88,  561, 8, 582, 24
84,  car, 3, 0.85,  269, 179, 446, 372
84,  car, 3, 0.85,  583, 2, 600, 17
84,  truck, 8, 0.85,  201, 27, 335, 123
84,  car, 3, 0.80,  653, 30, 691, 75
84,  person, 1, 0.80,  318, 92, 337, 121
84,  truck, 8, 0.79,  148, 90, 262, 179
84,  car, 3, 0.76,  51, 178, 163, 266
85,  car, 3, 0.93,  642, 31, 687, 71
85,  car, 3, 0.93,  264, 180, 448, 377
85,  potted plant, 59, 0.91,  37, 1, 243, 117
85,  car, 3, 0.88,  561, 8, 582, 24
85,  car, 3, 0.85,  583, 2, 600, 17
85,  truck, 8, 0.84,  201, 27, 335, 123
85,  car, 3, 0.82,  3, 365, 114, 536
85,  person, 1, 0.79,  318, 92, 337, 121
85,  truck, 8, 0.79,  148, 91, 262, 178
85,  car, 3, 0.74,  51, 178, 163, 266
86,  car, 3, 0.93,  642, 31, 687, 71
86,  potted plant, 59, 0.92,  37, 1, 243, 117
86,  car, 3, 0.88,  561, 8, 582, 24
86,  car, 3, 0.87,  258, 191, 439, 389
86,  truck, 8, 0.84,  201, 27, 335, 123
86,  car, 3, 0.84,  58

...

144,  potted plant, 59, 0.94,  45, 1, 235, 123
144,  car, 3, 0.88,  641, 38, 670, 71
144,  truck, 8, 0.82,  144, 92, 266, 176
144,  car, 3, 0.82,  228, 59, 335, 121
144,  car, 3, 0.80,  52, 179, 162, 267
144,  truck, 8, 0.79,  209, 29, 331, 120
144,  car, 3, 0.79,  560, 8, 582, 26
144,  person, 1, 0.70,  318, 92, 337, 121
145,  potted plant, 59, 0.96,  43, 1, 243, 123
145,  car, 3, 0.88,  641, 38, 670, 71
145,  car, 3, 0.84,  52, 179, 162, 267
145,  truck, 8, 0.83,  144, 93, 266, 176
145,  car, 3, 0.82,  227, 59, 335, 121
145,  truck, 8, 0.81,  210, 30, 331, 120
145,  car, 3, 0.80,  560, 8, 582, 26
145,  person, 1, 0.72,  318, 92, 337, 121
146,  potted plant, 59, 0.94,  46, 1, 237, 126
146,  car, 3, 0.87,  641, 38, 670, 71
146,  person, 1, 0.85,  598, 64, 615, 103
146,  car, 3, 0.84,  53, 179, 162, 267
146,  truck, 8, 0.83,  145, 93, 266, 176
146,  car, 3, 0.82,  228, 60, 335, 121
146,  truck, 8, 0.81,  209, 26, 325, 118
146,  car, 3, 0.78,  560, 8, 582, 26
147,  potted plant, 59, 0.94

167,  potted plant, 59, 0.95,  38, 5, 231, 122
167,  car, 3, 0.90,  643, 38, 668, 71
167,  car, 3, 0.84,  404, 33, 445, 62
167,  truck, 8, 0.84,  149, 92, 264, 174
167,  car, 3, 0.83,  54, 179, 161, 267
167,  car, 3, 0.82,  227, 58, 334, 121
167,  car, 3, 0.77,  651, 33, 690, 73
167,  truck, 8, 0.76,  200, 26, 334, 124
167,  car, 3, 0.75,  560, 7, 583, 25
167,  person, 1, 0.71,  605, 44, 617, 68
168,  car, 3, 0.91,  642, 39, 672, 71
168,  potted plant, 59, 0.89,  38, 5, 227, 143
168,  truck, 8, 0.86,  209, 25, 329, 121
168,  truck, 8, 0.85,  143, 89, 262, 179
168,  car, 3, 0.79,  655, 32, 694, 76
168,  car, 3, 0.77,  227, 59, 335, 121
168,  car, 3, 0.76,  52, 179, 162, 267
168,  person, 1, 0.75,  605, 44, 617, 68
168,  car, 3, 0.75,  560, 8, 582, 25
169,  car, 3, 0.90,  641, 39, 669, 72
169,  potted plant, 59, 0.88,  38, 5, 228, 147
169,  truck, 8, 0.86,  145, 89, 262, 179
169,  truck, 8, 0.85,  205, 28, 335, 122
169,  car, 3, 0.77,  52, 179, 162, 267
169,  car, 3, 0.77,  651, 31, 692,

191,  car, 3, 0.94,  642, 39, 669, 72
191,  potted plant, 59, 0.90,  35, 4, 232, 140
191,  truck, 8, 0.86,  148, 88, 263, 179
191,  car, 3, 0.84,  51, 178, 162, 267
191,  truck, 8, 0.82,  200, 27, 334, 122
191,  car, 3, 0.82,  650, 34, 692, 77
191,  person, 1, 0.82,  609, 31, 619, 50
191,  person, 1, 0.77,  315, 91, 337, 122
191,  car, 3, 0.71,  559, 8, 582, 28
192,  car, 3, 0.94,  642, 38, 670, 71
192,  potted plant, 59, 0.91,  42, 0, 236, 131
192,  truck, 8, 0.85,  147, 90, 267, 176
192,  truck, 8, 0.82,  198, 26, 335, 122
192,  car, 3, 0.81,  54, 179, 162, 267
192,  car, 3, 0.78,  654, 32, 691, 74
192,  car, 3, 0.77,  233, 60, 334, 121
192,  person, 1, 0.74,  313, 90, 337, 123
192,  person, 1, 0.74,  609, 30, 619, 49
193,  car, 3, 0.94,  642, 38, 670, 71
193,  potted plant, 59, 0.90,  36, 6, 230, 130
193,  truck, 8, 0.85,  147, 91, 263, 178
193,  person, 1, 0.84,  609, 30, 619, 49
193,  car, 3, 0.83,  54, 179, 162, 267
193,  truck, 8, 0.80,  206, 24, 323, 118
193,  car, 3, 0.79,  24

215,  car, 3, 0.93,  642, 35, 676, 71
215,  potted plant, 59, 0.91,  41, 2, 227, 133
215,  car, 3, 0.83,  53, 178, 162, 266
215,  truck, 8, 0.83,  202, 30, 333, 122
215,  truck, 8, 0.83,  147, 93, 265, 175
215,  car, 3, 0.80,  561, 7, 582, 23
215,  person, 1, 0.75,  314, 91, 337, 123
215,  person, 1, 0.74,  613, 25, 622, 39
215,  car, 3, 0.71,  242, 61, 333, 120
216,  car, 3, 0.94,  643, 39, 673, 71
216,  truck, 8, 0.91,  148, 92, 265, 176
216,  potted plant, 59, 0.89,  44, 1, 219, 130
216,  truck, 8, 0.89,  202, 28, 332, 121
216,  car, 3, 0.79,  407, 36, 442, 60
216,  car, 3, 0.72,  55, 179, 162, 267
217,  car, 3, 0.94,  643, 38, 674, 71
217,  truck, 8, 0.91,  147, 92, 265, 176
217,  truck, 8, 0.91,  203, 25, 331, 120
217,  potted plant, 59, 0.84,  39, 1, 239, 120
217,  car, 3, 0.78,  407, 36, 442, 59
217,  car, 3, 0.71,  55, 179, 162, 267
218,  car, 3, 0.94,  643, 39, 673, 71
218,  truck, 8, 0.91,  202, 27, 331, 122
218,  truck, 8, 0.89,  147, 92, 265, 176
218,  potted plant, 59, 0.8

243,  car, 3, 0.93,  401, 28, 444, 56
243,  potted plant, 59, 0.92,  37, 1, 227, 136
243,  car, 3, 0.89,  642, 34, 678, 65
243,  truck, 8, 0.84,  153, 83, 265, 172
243,  car, 3, 0.81,  51, 171, 163, 263
244,  truck, 8, 0.92,  151, 84, 263, 174
244,  car, 3, 0.91,  48, 170, 161, 262
244,  car, 3, 0.85,  641, 34, 672, 59
244,  potted plant, 59, 0.84,  35, 1, 220, 129
244,  car, 3, 0.82,  639, 46, 655, 64
244,  person, 1, 0.76,  216, 158, 234, 186
244,  truck, 8, 0.76,  209, 37, 328, 121
244,  car, 3, 0.73,  399, 28, 439, 57
245,  potted plant, 59, 0.94,  31, 2, 215, 126
245,  car, 3, 0.93,  45, 175, 156, 263
245,  car, 3, 0.89,  639, 32, 679, 64
245,  truck, 8, 0.87,  148, 75, 260, 174
245,  car, 3, 0.83,  551, 1, 578, 23
245,  truck, 8, 0.79,  196, 28, 320, 123
245,  car, 3, 0.75,  383, 26, 436, 55
245,  car, 3, 0.73,  636, 43, 655, 65
245,  car, 3, 0.70,  649, 27, 711, 75
246,  potted plant, 59, 0.93,  34, 2, 214, 148
246,  car, 3, 0.90,  45, 175, 155, 264
246,  car, 3, 0.86,  397, 27,

### Compute mAP @ IoU=50 on Batch of Images

### Precision-Recall

In [ ]:
'''
#print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))
#dataset.image_info[1:3]

print(dataset.num_classes)
print(dataset.num_images)
print(len(dataset.image_info[2]))
print(dataset.image_info[2]['id'])
print(dataset.image_info[2]['source'])
print(dataset.image_info[2]['path'])
print(dataset.image_info[2]['width'])
print(dataset.image_info[2]['height'])
#print(dataset.image_info[2]['annotations'])
#print(dataset.image_info[2])
print('')

print(dataset.class_names[8])
print(dataset.class_ids)
print('')

print(dataset.class_info[80])
print(dataset.class_info[80]['source'])
dataset.class_info[80]['name']
'''